In [7]:
import pandas as pd
import numpy as np
import string
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [8]:
f_path = '/content/Fake.csv'
r_path = '/content/True.csv'
data_fake = pd.read_csv(f_path)
data_true = pd.read_csv(r_path)
data_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [9]:
data_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [10]:
data_fake["class"] = 0
data_true['class'] = 1

In [11]:
data_fake_manual_testing = data_fake.tail(10).copy()
for i in range(23480, 23470, -1):
  data_fake.drop([i], axis=0, inplace=True)
data_true_manual_testing = data_true.tail(10).copy()
for i in range(21416, 23405, -1):
  data_true.drop([i], axis=0, inplace=True)

In [12]:
data_fake_manual_testing['class'] = 0
data_true_manual_testing['class'] = 1

In [13]:
data_fake_manual_testing.head()

,title,text,subject,date,class
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0


In [14]:
data_merge = pd.concat([data_fake, data_true], axis=0)
data_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [15]:
data_merge = data_merge.drop(['title', 'subject', 'date'], axis=1)


In [17]:
data_merge.isnull().sum()

,0
text,0
class,0


In [18]:
data_merge=data_merge.sample(frac=1)
data_merge.head()

,text,class
12719,JUBA (Reuters) - At least 45 people have died ...,1
15105,Patriotic University of Missouri students burn...,0
18487,Nancy Pelosi attempted to offer her (incoheren...,0
13772,BERLIN (Reuters) - During Germany s election c...,1
9870,WASHINGTON (Reuters) - The secretive U.S. Fore...,1


In [19]:
data_merge.reset_index(inplace=True)
data_merge.drop(['index'], axis=1, inplace=True)
data_merge.head()

,text,class
0,JUBA (Reuters) - At least 45 people have died ...,1
1,Patriotic University of Missouri students burn...,0
2,Nancy Pelosi attempted to offer her (incoheren...,0
3,BERLIN (Reuters) - During Germany s election c...,1
4,WASHINGTON (Reuters) - The secretive U.S. Fore...,1


In [20]:
def wordopt(text):
  text = text.lower()
  text = re.sub(r'\[.*?\]', '', text)
  text = re.sub(r"\\W", " ", text)
  text = re.sub(r'https?://\S+|www\.\S+', '', text)
  text = re.sub(r'<.*?>+', '', text)
  text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
  text = re.sub(r'\n', '', text)
  text = re.sub(r'\w*\d\w*', '', text)
  return text


In [21]:
data_merge['text'] = data_merge['text'].apply(wordopt)


In [22]:
x = data_merge['text']
y = data_merge['class']


In [23]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)


In [25]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [27]:
LR = LogisticRegression()
#DT = DecisionTreeClassifier()
LR.fit(xv_train, y_train)
#DT.fit(xv_train, y_train)


LogisticRegression()

In [28]:
pred_lr=LR.predict(xv_test)
#pred_dt=DT.predict(xv_test)

In [29]:
LR.score(xv_test, y_test)

0.9878809481375869

In [31]:
!pip install scikit-learn
from sklearn.metrics import classification_report

print(classification_report(y_test, pred_lr)) # Now the function is available

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5879
           1       0.99      0.99      0.99      5343

    accuracy                           0.99     11222
   macro avg       0.99      0.99      0.99     11222
weighted avg       0.99      0.99      0.99     11222



In [32]:
#LR = LogisticRegression()
DT = DecisionTreeClassifier()
#LR.fit(xv_train, y_train)
DT.fit(xv_train, y_train)


DecisionTreeClassifier()

In [33]:
pred_dt=DT.predict(xv_test)

In [34]:
DT.score(xv_test, y_test)

0.9960791302798075

In [37]:
from sklearn.ensemble import GradientBoostingClassifier
GB=GradientBoostingClassifier(random_state=0)
GB.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [38]:
predict_gb=GB.predict(xv_test)

In [39]:
GB.score(xv_test, y_test)

0.9952771342006772

In [40]:
print(classification_report(y_test, predict_gb))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5879
           1       0.99      1.00      1.00      5343

    accuracy                           1.00     11222
   macro avg       1.00      1.00      1.00     11222
weighted avg       1.00      1.00      1.00     11222



In [41]:
from sklearn.ensemble import RandomForestClassifier
RF=RandomForestClassifier(random_state=0)
RF.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [42]:
predict_rf=RF.predict(xv_test)

In [43]:
RF.score(xv_test, y_test)

0.9887720548921761

In [44]:
print(classification_report(y_test, predict_rf))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5879
           1       0.99      0.99      0.99      5343

    accuracy                           0.99     11222
   macro avg       0.99      0.99      0.99     11222
weighted avg       0.99      0.99      0.99     11222



In [45]:
def output_label(n):
  if n == 0:
    return "Fake News"
  elif n == 1:
    return "Not a Fake News"
# Manual testing function
def manual_testing(news):
  testing_news = {"text": [news]}
  new_def_test = pd.DataFrame(testing_news)
  new_def_test["text"] = new_def_test["text"].apply(wordopt)
  new_x_test = new_def_test["text"]
  new_xv_test = vectorization.transform(new_x_test)
  pred_lr = LR.predict(new_xv_test)
  pred_dt = DT.predict(new_xv_test)
  return (output_label(pred_lr[0]),output_label(pred_dt[0]),)


In [48]:
def main():
  st.title('FAKE CIRCULAR DETECTOR')
  input_text = st.text_input("Enter the news article")
  if st.button("Check for Fake News"):
    st.text("Classifying...")
# Process the input text
    input_text_processed = wordopt(input_text)
    new_x_test = pd.Series(input_text_processed)
    new_xv_test = vectorization.transform(new_x_test)
# Make predictions
    pred_lr = LR.predict(new_xv_test)
    pred_dt = DT.predict(new_xv_test)
    st.write("- LR Prediction: {}".format(output_label(pred_lr[0])))
    st.write("- DT Prediction: {}".format(output_label(pred_dt[0])))
  if __name__ == '__main__':
    main()

In [49]:
news=str(input())
manual_testing(news)

news=str(input()) manual_testing(news)


('Fake News', 'Fake News')